In [1]:
import GetOldTweets3 as got
import os
import pandas as pd
import numpy as np
import datetime
import re

In [2]:
os.chdir("C:\\Users\\aliss\\Documents\\Nova IMS\\Cursos\\Text Mining\\Final Project")

In [95]:
tweetCriteria = got.manager.TweetCriteria().setQuerySearch("Bolsonaro candidato lang:pt")\
                                           .setSince("2018-08-10")\
                                           .setUntil("2018-10-10")\
                                           .setMaxTweets(5)
tweets = got.manager.TweetManager.getTweets(tweetCriteria)

In [96]:
for tweet in tweets:
    print(tweet.text, "\n")

“Apoiadores do Candidato à presidência Bolsonaro...Agride à oposição” #HaddadSim #EleNao #Hadadpresidente passem láhttps://twitter.com/i/moments/1049807892201832449 … 

Você manda um vídeo do PRÓPRIO CANDIDATO FALANDO que vai governar para maiorias e o sujeito fala "A mas isso não ta escrito na proposta dele, não vai ser assim". Alô Bolsonaro, você pode cagar na boca do seu eleitorado que eles vão comer falando que é brigadeiro. Desisto. 

O Bolsonaro diz que não vai apoiar o candidato do PSL em SC e quer apoio do NOVO.? https://twitter.com/reutersbrasil/status/1049747031051444224?s=21 … 

Coragem e força a vitória virar jamais um candidato perdeu com ampla vantagem com.tem Horário eleitoral mostrará 'história vitoriosa de Bolsonaro' https://www.terra.com.br/noticias/eleicoes/programa-da-tv-do-psl-mostrara-historia-vitoriosa-de-bolsonaro-diz-filho,3542ade88f35225ad3c8f22f0349f1bb74tn4gi5.html … 

Eu também não concordo! Mas também acho que alguém envolvido na vida política declarando a

In [97]:
n = len(tweets)

aux_id = [0] * n
aux_text = [" "] * n
aux_ht = [" "] * n
aux_date = [datetime.date.today()] * n

for i in range(n):
    aux_id[i] = tweets[i].id
    aux_text[i] = tweets[i].text
    aux_ht[i] = tweets[i].hashtags
    aux_date[i] = tweets[i].date
    
df = pd.DataFrame(np.transpose([aux_id, aux_text, aux_ht, aux_date]), columns=["id", "text", " hashtags", "date"])

In [ ]:
#df.to_csv("2019_02_05-2019_03_05.csv")
#df.to_csv("2018_09_10-2018_10_10.csv")

#### 2 - Processamento dos Tweets

##### Dúvidas:
* O que fazer com tweets de notícia?
* Verifica a situação do stopwords;
* Como tratar as abreviações?

##### 3 - Pré Processamento
    1 - Remoção de links, pois esses termos não possuem conteúdo semântico;
    2 - Substituição de caracteres acentuados pelos correspondentes sem acentuação;
    3 - 
    3 - Remoção de repetição de letras;
    4 - Remoção de citação a outros usuários;
    5 - Conversão de letras maiúsculas em minúsculas, a fim de padronizar o texto;
    6 - Remoção de caracteres não alfabéticos e pontuação;
    7 - Remoção de caracteres numéricos;

In [99]:
# Use o cmd to install: conda install -c conda-forge spacy
# baixar o modelo em português: python -m spacy download pt
import spacy
from unicodedata import normalize
nlp = spacy.load("pt")

In [ ]:
# Verificando se NÃO é um stop word
#nlp.vocab['não'].is_stop
# Removendo NÃO da lista de stopwords
#nlp.Defaults.stop_words.remove('beyond')

In [104]:
def bigrama_nao(texto):
    p = re.compile(r'(nao) (\w*)')
    bign = p.findall(texto)
    for i in range(len(bign)):
        texto = re.sub(str(bign[i][0]+' '+bign[i][1]), str(bign[i][0]+'_'+bign[i][1]), texto)

    return texto

In [105]:
# Pré processamento

def pre_process(df):
    df['text'] = df['text'].apply(lambda x: re.sub(r"http\S+", "",x)) # 1
    df['text'] = df['text'].apply(lambda x: normalize('NFKD', x).encode('ASCII', 'ignore').decode('ASCII')) # 2
    df['text'] = df['text'].apply(lambda x: re.sub(r'\bn\b', 'nao',x, flags=re.IGNORECASE)) # 
    df['text'] = df['text'].apply(lambda x: re.sub(r'(\w)(\1{2,})', r'\1',x)) # 3
    df['text'] = df['text'].apply(lambda x: re.sub(r'(\B[@](\w* ))', '',x)) # 4
    df['text'] = df['text'].apply(lambda x: x.lower()) # 5
    df['text'] = df['text'].apply(lambda x: re.sub(r'[\W*]+', ' ',x)) # 6
    df['text'] = df['text'].apply(lambda x: re.sub(r'[0-9]', '',x)) # 7
    df['text'] = df['text'].apply(lambda x: bigrama_nao(x)) # 7
    
    # Abreviações
    df['text'] = df['text'].apply(lambda x: re.sub(r'\bpq\b', 'porque',x)) 
    df['text'] = df['text'].apply(lambda x: re.sub(r'\bvc\b', 'voce',x)) 
    df['text'] = df['text'].apply(lambda x: re.sub(r'\btb\b', 'tambem',x)) 
    df['text'] = df['text'].apply(lambda x: re.sub(r'\btbm\b', 'tambem',x)) 

    return df    

In [106]:
# testando função de pre processamento
pre_process(df)

,id,text,hashtags,date
0,1049811735715213314,apoiadores do candidato a presidencia bolsonar...,#HaddadSim #EleNao #Hadadpresidente,2018-10-09 23:59:57+00:00
1,1049811735715155968,voce manda um video do proprio candidato falan...,,2018-10-09 23:59:57+00:00
2,1049811670120505344,o bolsonaro diz que nao_vai apoiar o candidato...,,2018-10-09 23:59:41+00:00
3,1049811666802810880,coragem e forca a vitoria virar jamais um cand...,,2018-10-09 23:59:40+00:00
4,1049811649975242752,eu tambem nao_concordo mas tambem acho que alg...,,2018-10-09 23:59:36+00:00


In [107]:
print( df['text'])

0    apoiadores do candidato a presidencia bolsonar...
1    voce manda um video do proprio candidato falan...
2    o bolsonaro diz que nao_vai apoiar o candidato...
3    coragem e forca a vitoria virar jamais um cand...
4    eu tambem nao_concordo mas tambem acho que alg...
Name: text, dtype: object


#### Ideias

* Tf-idf not only counts the occurrence of a word in the given text(or document), but also reflect how important the word is to the document. 
* Try with "bigram"